# Process Automation Exmaple
### Dataset comes from Kaggle 
#### https://www.kaggle.com/mirichoi0218/insurance

##### Columns
**age:** age of primary beneficiary

**sex:** insurance contractor gender, female, male

**bmi:** Body mass index, providing an understanding of body, weights that are relatively high or low relative to height,
objective index of body weight (kg / m ^ 2) using the ratio of height to weight, ideally 18.5 to 24.9

**children:** Number of children covered by health insurance / Number of dependents

**smoker:** Smoking

**region:** the beneficiary's residential area in the US, northeast, southeast, southwest, northwest.

**charges:** Individual medical costs billed by health insurance

In [1]:
#Import packages
import pandas as pd
import numpy as np
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import requests
import io

# Downloading the csv file from your GitHub account

url = "https://raw.githubusercontent.com/maulcait/Python-Practice-Applications/main/insurance.csv" 
# Make sure the url is the raw version of the file on GitHub
download = requests.get(url).content

# Reading the downloaded content and turning it into a pandas dataframe

df = pd.read_csv(io.StringIO(download.decode('utf-8')))
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


### Hypothetical example: 
Let's pretend that we want all of the data in separate csv files by age

We could just go in an copy paste each age group into a separate file and save as a csv

or... 

We could use python to break out each age group into separate csv files

Let's try the python way.


In [2]:
#Import the os package to help with manipulating files and directories
import os
#Create a new folder to hold all the files for this example
examplefolder = 'python_process_automation_example'
if not os.path.isdir(examplefolder):
       os.makedirs(examplefolder)
#change working directory so that all future folders and files created will go into the example folder we just created
os.chdir(examplefolder)


In [3]:
#Create a new folder to hold all the files
folderName = 'insurance_by_age'
if not os.path.isdir(folderName):
       os.makedirs(folderName)

In [4]:
#Loop through the unique ages and write out a csv file for each one
#exclude the index column when writing out the files
for i in df['age'].unique():
    subset_df = df[df['age'] == i]
    (subset_df).to_csv(folderName+ '/'+'insurance_age_'+str(i) + '.csv',index=False)

#### Let's pretend now that we were starting with this folder of CSV files and we wanted to consolidate all these files into a single excel document where each tab is an age group and the data associated with that age group

We will have three different age group buckets:

Young adults (18-30)

Adult (31 - 50) 

Older Adults (51+)

We would like to group all the ages and put them on a specific tab

In [5]:
#Create the folder and excel file that will hold all the data
#Create a new folder to hold all the files
excelFolderName = 'insurance_by_age_group'
if not os.path.isdir(excelFolderName):
       os.makedirs(excelFolderName)       


In [6]:
#import package to help search for all the csv files
import glob
#get all the filenames in the folder
fileList = glob.glob(folderName+"/*.csv")
#display the first 5 records
fileList[0:5]

['insurance_by_age\\insurance_age_18.csv',
 'insurance_by_age\\insurance_age_19.csv',
 'insurance_by_age\\insurance_age_20.csv',
 'insurance_by_age\\insurance_age_21.csv',
 'insurance_by_age\\insurance_age_22.csv']

In [7]:
#create a dataframe to hold all the file data
allFilesDf = pd.DataFrame()
#loop through list of files and read into a dataframe
for i in fileList:
    allFilesDf = allFilesDf.append(pd.read_csv(i))

In [8]:
#display top 5 records of dataframe 
allFilesDf.head()

,age,sex,bmi,children,smoker,region,charges
0,18,male,33.770,1,no,southeast,1725.55230
1,18,male,34.100,0,no,southeast,1137.01100
2,18,female,26.315,0,no,northeast,2198.18985
3,18,female,38.665,2,no,northeast,3393.35635
4,18,female,35.625,0,no,northeast,2211.13075


In [9]:
#Create the excel file with all the data on one tab
allFilesDf.to_excel(excelFolderName+'/all_data.xlsx')

In [10]:
#add column for sheetname
allFilesDf['sheetname'] = np.where(allFilesDf['age'] <=30, 'young adult', np.where(allFilesDf['age'] <=50, 'adult', 'older adult'))

In [11]:
#iterate through unique sheet name values to write to excel worksheet

#create a Pandas Excel writer using XlsxWriter as the engine
writer = pd.ExcelWriter(excelFolderName+'/insurance_by_age_group.xlsx', engine='xlsxwriter')

#write each DataFrame to a specific sheet
for i in allFilesDf['sheetname'].unique():
    allFilesDf[allFilesDf['sheetname'] == i].to_excel(writer, sheet_name=i,index=False)
#close the Pandas Excel writer and output the Excel file
writer.save()

#### Now that we have saved the files into the excel document lets archive all the old csv files

In [12]:
#Create an archive folder if it doesn't exist
archiveFolderName = excelFolderName+'/archive'
if not os.path.isdir(archiveFolderName):
       os.makedirs(archiveFolderName)      

In [27]:
#move all the individual csv files into the archive folder
filesToArchive = os.listdir(folderName)
print("Files to archive: " + str(len(filesToArchive)))

Files to archive: 0


In [28]:
#iterate through list of files and rename to the archive filepath
for f in filesToArchive:
    os.rename(folderName+ '/'  + f, archiveFolderName+ '/' + f)

In [30]:
#verify this operation worked
print("Files archived: " + str(len(filesToArchive)))

Files archived: 0


#### Next steps might include:

checking the folder for new files

scheduling this program to run at specific intervals

writing the data to a database table

creating some custom visualization that update as new data gets pulled in 

applying a machine learning model to the new data 